In [2]:
# Cell 1: Install dependencies
!pip install ultralytics opencv-python-headless


In [3]:

# Cell 2: Imports
import cv2
import base64
import numpy as np
import time
from ultralytics import YOLO
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from IPython.display import display, Javascript, HTML


In [4]:
# Cell 3: Initialize webcam (RUN THIS FIRST!)
def setup_webcam():
    """Initialize webcam access - must run this cell first!"""
    js_code = Javascript('''
    async function setupCamera() {
        const div = document.createElement('div');
        div.innerHTML = `
            <div style="padding: 20px; background: #f0f0f0; border-radius: 10px;">
                <h3>📷 Webcam Setup</h3>
                <video id="webcam" autoplay playsinline style="display:none;"></video>
                <p id="status">Initializing camera...</p>
            </div>
        `;
        document.body.appendChild(div);

        try {
            const stream = await navigator.mediaDevices.getUserMedia({
                video: { width: 640, height: 480 }
            });
            const video = document.getElementById('webcam');
            video.srcObject = stream;
            await video.play();
            document.getElementById('status').innerHTML = '✓ Camera ready! You can now run detection.';
            document.getElementById('status').style.color = 'green';
        } catch(err) {
            document.getElementById('status').innerHTML = '✗ Error: ' + err.message;
            document.getElementById('status').style.color = 'red';
        }
    }
    setupCamera();
    ''')
    display(js_code)
    print("⚠️ Please allow camera access when prompted!")
    print("⚠️ Wait for 'Camera ready' message before proceeding")


In [5]:
# Cell 4: Improved frame capture function
def capture_frame():
    """Capture frame from webcam"""
    js_code = '''
    async function captureFrame() {
        try {
            const video = document.createElement('video');
            const stream = await navigator.mediaDevices.getUserMedia({
                video: { width: 640, height: 480, facingMode: 'user' }
            });

            video.srcObject = stream;
            video.setAttribute('playsinline', true);
            await video.play();

            // Wait for video to be fully ready
            await new Promise(r => setTimeout(r, 500));

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;

            const ctx = canvas.getContext('2d');
            ctx.drawImage(video, 0, 0, canvas.width, canvas.height);

            // Stop all tracks
            stream.getTracks().forEach(track => track.stop());

            return canvas.toDataURL('image/jpeg', 0.95);

        } catch(err) {
            console.error('Capture error:', err);
            return null;
        }
    }
    '''

    try:
        # Execute JavaScript and get result
        data = eval_js(js_code + '\ncaptureFrame()')

        # Check if data is valid
        if data is None:
            print("⚠️ eval_js returned None - camera might not be initialized")
            print("💡 Try running setup_webcam() first!")
            return None

        if not isinstance(data, str):
            print(f"⚠️ Unexpected data type: {type(data)}")
            return None

        if not data.startswith('data:image'):
            print(f"⚠️ Invalid data format: {data[:50]}...")
            return None

        # Decode base64 image
        img_data = base64.b64decode(data.split(',')[1])
        frame = cv2.imdecode(np.frombuffer(img_data, np.uint8), cv2.IMREAD_COLOR)

        if frame is None:
            print("⚠️ Failed to decode image")
            return None

        return frame

    except Exception as e:
        print(f"❌ Capture error: {e}")
        import traceback
        traceback.print_exc()
        return None


In [6]:



# Cell 5: Test capture function
def test_capture():
    """Test if camera capture is working"""
    print("Testing camera capture...")
    frame = capture_frame()

    if frame is not None:
        print(f"✓ Success! Captured frame: {frame.shape}")
        cv2_imshow(frame)
        return True
    else:
        print("✗ Capture failed")
        print("\n📋 Troubleshooting steps:")
        print("1. Make sure you ran setup_webcam() and allowed camera access")
        print("2. Check if you see 'Camera ready' message")
        print("3. Try refreshing the page and running cells again")
        print("4. Some browsers block camera in Colab - try Chrome")
        return False



In [ ]:
# Cell 6: Load YOLO model
try:
    print("Loading YOLO model...")
    model = YOLO("yolov8n.pt")
    print("✓ Model loaded successfully!")
    print(f"Can detect {len(model.names)} classes: {list(model.names.values())[:8]}...")
except Exception as e:
    print(f" Error loading model: {e}")



Loading YOLO model...
✓ Model loaded successfully!
Can detect 80 classes: ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat']...


In [8]:
# Cell 7: Improved Detection function
def detect_objects(frame, model, conf_threshold=0.25, iou_threshold=0.45):
    """
    Perform object detection on frame with better parameters

    Args:
        conf_threshold: Lower = more detections (0.25 recommended)
        iou_threshold: Controls box overlap filtering
    """
    if frame is None:
        return None

    # Run detection with optimized parameters
    results = model(frame, conf=conf_threshold, iou=iou_threshold,
                   verbose=False, imgsz=640)

    detection_count = 0
    detected_objects = []

    for r in results:
        boxes_data = []

        # Collect all detections first
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])
            label = model.names[int(box.cls[0])]

            boxes_data.append({
                'coords': (x1, y1, x2, y2),
                'conf': conf,
                'label': label
            })

        # Sort by y-coordinate to stack labels properly
        boxes_data.sort(key=lambda x: x['coords'][1])

        # Draw each detection
        for data in boxes_data:
            detection_count += 1
            x1, y1, x2, y2 = data['coords']
            conf = data['conf']
            label = data['label']

            detected_objects.append(f"{label}({conf:.2f})")

            # Better color coding
            if conf > 0.7:
                color = (0, 255, 0)  # Green - high confidence
            elif conf > 0.5:
                color = (0, 200, 255)  # Orange - medium confidence
            else:
                color = (0, 100, 255)  # Red - low confidence

            # Draw thinner, cleaner box
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

            # Cleaner label with better positioning
            text = f"{label} {conf:.0%}"
            font_scale = 0.5
            thickness = 1
            (tw, th), baseline = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX,
                                                  font_scale, thickness)

            # Position label above box, or below if too close to top
            if y1 - th - 8 < 0:
                label_y = y2 + th + 8
                rect_y1, rect_y2 = y2, y2 + th + 8
            else:
                label_y = y1 - 4
                rect_y1, rect_y2 = y1 - th - 8, y1

            # Draw label background with padding
            cv2.rectangle(frame, (x1, rect_y1), (x1 + tw + 8, rect_y2), color, -1)

            # Draw text in white
            cv2.putText(frame, text, (x1 + 4, label_y),
                       cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), thickness)

    # Compact info overlay in corner
    info = f"Found: {detection_count}"
    cv2.rectangle(frame, (5, 5), (150, 35), (0, 0, 0), -1)
    cv2.putText(frame, info, (10, 25),
               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    if detected_objects:
        print(f"  Found: {', '.join(detected_objects)}")

    return frame


In [ ]:

# Cell 8: Run detection with better defaults
def run_detection(num_frames=5, conf_threshold=0.25, delay=1.0):
    """
    Run object detection loop

    Args:
        num_frames: Number of frames to capture
        conf_threshold: 0.25 recommended (lower = more detections)
        delay: Seconds between frames
    """
    print(f"\n{'='*60}")
    print(f"Starting detection: {num_frames} frames, confidence={conf_threshold}")
    print(f"{'='*60}\n")

    successful = 0
    failed = 0

    for i in range(num_frames):
        print(f"📸 Frame {i+1}/{num_frames}...", end=" ")

        frame = capture_frame()

        if frame is None:
            print("S Capture failed")
            failed += 1
            time.sleep(delay)
            continue

        print(f"✓ Captured ({frame.shape[1]}x{frame.shape[0]})")

        annotated = detect_objects(frame, model, conf_threshold)

        if annotated is not None:
            cv2_imshow(annotated)
            successful += 1

        if i < num_frames - 1:  # Don't sleep after last frame
            time.sleep(delay)

    print(f"\n{'='*60}")
    print(f"✓ Completed: {successful} successful, {failed} failed")
    print(f"{'='*60}")

In [1]:
run_detection(num_frames=10, conf_threshold=0.5, delay=0.5)

NameError: name 'run_detection' is not defined